In [ ]:
# Install required libraries
%pip install ipykernel
%pip install scipy
%pip install pandas
%pip install nltk
%pip install gensim
%pip install transformers torch

# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 20.0 MB/s eta 0:00:00 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

print("Environment setup complete!")

# File paths
filepaths = [
    'organized_blow_molding_basics.csv',
    'Dupont_Table7.csv',
    'organized_pet_bottle_faults.csv',
    'organized_stressesinpebottles.csv'
]

# Potential column names to search for
potential_columns = ["text", "Fault","Cause","Solution", "contents","description","cause","solution"]

# Function to find a matching column in a DataFrame
def find_matching_column(df, potential_columns):
    for column in potential_columns:
        if column in df.columns:
            return column
    return None

# Function to load and concatenate text data from each CSV file
def load_text_from_csv(filepaths, potential_columns):
    text_data = []
    for filepath in filepaths:
        df = pd.read_csv(filepath)
        # Find a matching column
        matching_column = find_matching_column(df, potential_columns)
        if matching_column:
            print(f"Using column '{matching_column}' from {filepath}")
            text_data.extend(df[matching_column].dropna().tolist())
        else:
            print(f"No matching columns found in {filepath}. Skipping this file.")
    return text_data

# Load and concatenate text data from CSV files
texts = load_text_from_csv(filepaths, potential_columns)

# If no text data is found, exit the script
if not texts:
    raise ValueError("No text data found. Please ensure your CSV files contain at least one of the specified columns.")

# Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

# Preprocess all text data
processed_texts = [preprocess_text(text) for text in texts]

# Word2Vec embeddings
word2vec_model = Word2Vec(sentences=processed_texts, vector_size=100, window=5, min_count=1, workers=4)
print("Word2Vec Embedding for 'example':", word2vec_model.wv['example'])  # Replace 'example' with a word in your dataset

# BERT embeddings using Hugging Face's transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = bert_model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Sentence-level embedding
    return embeddings

# Generate BERT embeddings for each preprocessed text
bert_embeddings = [get_bert_embeddings(" ".join(tokens)) for tokens in processed_texts]

# Display an example
print("BERT Embedding for first document:", bert_embeddings[0].detach().numpy())

NameError: name 'nltk' is not defined